In [ ]:
%load_ext autoreload
%autoreload 2
import os
from dimcat import (
    Corpus,  
    Pipeline,
    IsAnnotatedFilter,
    CorpusGrouper, 
    PieceGrouper, 
    ModeGrouper, 
    ChordSymbolBigrams, 
    ChordSymbolUnigrams,
    LocalKeySlicer,
)
from dimcat import __version__ as dimcat_version
from ms3 import __version__ as ms3_version
from git import Repo
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
pd.set_option("display.max_columns", 100)

In [ ]:
corpus_path = "~/romantic_piano_corpus"

repo = Repo(corpus_path)
print(f"{os.path.basename(corpus_path)} @ {repo.commit().hexsha[:7]}")
print(f"dimcat version {dimcat_version}")
print(f"ms3 version {ms3_version}")

# Overview

In [ ]:
corpus = Corpus(directory=corpus_path)
corpus.data

## Metadata

In [ ]:
all_metadata = corpus.data.metadata(from_tsv=True)
print(f"Concatenated 'metadata.tsv' files cover {len(all_metadata)} of the {len(corpus.data._score_ids())} scores.")
all_metadata.groupby(level=0).nth(0)

In [ ]:
print("VALUE COUNTS OF THE COLUMN 'annotators'")
all_metadata.annotators.value_counts()

In [ ]:
print(f"Composition dates range from {all_metadata.composed_start.min()} ({all_metadata.loc[all_metadata.composed_start.idxmin(), 'fnames']}) "
      f"to {all_metadata.composed_end.max()} ({all_metadata.loc[all_metadata.composed_end.idxmax(), 'fnames']}).")

In [ ]:
annotated = IsAnnotatedFilter().process_data(corpus)
print(f"Before: {len(corpus.indices[()])} IDs, after filtering: {len(annotated.indices[()])}")

**Choose here if you want to see stats for all or only for annotated scores.**

In [ ]:
#selected = corpus
selected = annotated

## Measures

In [ ]:
all_measures = selected.get_facet('measures')
print(f"{len(all_measures.index)} measures over {len(all_measures.groupby(level=[0,1]))} files.")
all_measures.head()

In [ ]:
print("Distribution of time signatures per XML measure (MC):")
all_measures.timesig.value_counts(dropna=False)

## Notes

In [ ]:
all_notes = selected.get_facet('notes')
print(f"{len(all_notes.index)} notes over {len(all_notes.groupby(level=[0,1]))} files.")
all_notes.head()

In [ ]:
STD_LAYOUT = {
 'paper_bgcolor': '#FFFFFF',
 'plot_bgcolor': '#FFFFFF',
 'margin': {'l': 40, 'r': 0, 'b': 0, 't': 40, 'pad': 0},
 'font': {'size': 15}
}

In [ ]:
def weight_notes(nl, group_col='midi', precise=True):
    summed_durations = nl.groupby(group_col).duration_qb.sum()
    summed_durations /= summed_durations.min()
    if not precise:
        # This simple trick reduces compute time but also precision:
        # The rationale is to have the smallest value be slightly larger than 0.5 because
        # if it was exactly 0.5 it would be rounded down by repeat_notes_according_to_weights()
        summed_durations /= 1.9999999
    return repeat_notes_according_to_weights(summed_durations)
    
def repeat_notes_according_to_weights(weights):
    counts = weights.round().astype(int)
    counts_reflecting_weights = []
    for pitch, count in counts.iteritems():
        counts_reflecting_weights.extend([pitch]*count)
    return pd.Series(counts_reflecting_weights)

grouped_notes = all_notes.groupby(level=0)
weighted_midi = pd.concat([weight_notes(nl, 'midi', precise=False) for _, nl in grouped_notes], axis=1, keys=grouped_notes.groups.keys())
weighted_midi

In [ ]:
yaxis=dict(tickmode= 'array',
           tickvals= [12, 24, 36, 48, 60, 72, 84, 96],
           ticktext = ["C0", "C1", "C2", "C3", "C4", "C5", "C6", "C7"],
           gridcolor='lightgrey',
           )
fig = px.violin(weighted_midi, labels=dict(variable='', value='pitch'), box=True, height=500) #, title="Distribution of pitches per corpus"
fig.update_layout(yaxis=yaxis, **STD_LAYOUT)
fig.write_image("/home/hentsche/Documents/phd/romantic_piano_corpus_report/figures/ambitus_per_corpus.png", scale=2)
fig.show()

In [ ]:
weighted_tpc = pd.concat([weight_notes(nl, 'tpc') for _, nl in grouped_notes], axis=1, keys=grouped_notes.groups.keys())
weighted_tpc

In [ ]:
yaxis=dict(
    tickmode= 'array',
    tickvals= [-12, -9, -6, -3, 0, 3, 6, 9, 12, 15, 18],
    ticktext = ["Dbb", "Bbb", "Gb", "Eb", "C", "A", "F#", "D#", "B#", "G##", "E##"],
    gridcolor='lightgrey',
    zerolinecolor='lightgrey',
    zeroline=True
           )
fig = px.violin(weighted_tpc, labels=dict(variable='', value='pitch class'), box=True, height=500)
fig.update_layout(yaxis=yaxis, **STD_LAYOUT)
fig.write_image("/home/hentsche/Documents/phd/romantic_piano_corpus_report/figures/tpc_per_corpus.png", scale=2)
fig.show()

In [ ]:
# adapted from https://plotly.com/python/violin/#ridgeline-plot
fig = go.Figure()
for corpus, data_line in weighted_tpc.iteritems():
    fig.add_trace(go.Violin(x=data_line, name=corpus))

fig.update_traces(side='positive', orientation='h', width=2, points=False)
fig.update_layout(xaxis_showgrid=False, xaxis_zeroline=True, height=600)
fig.show()

In [ ]:
fig = make_subplots(rows=len(grouped_notes), cols=1, subplot_titles=list(grouped_notes.groups.keys()), shared_xaxes=True)
for i, (corpus, notes) in enumerate(grouped_notes, 1):
    tpc_durations = notes.groupby('tpc').duration_qb.sum()
    tpc_durations /= tpc_durations.sum()
    fig.add_trace(go.Scatter(x=tpc_durations.index, y=tpc_durations, name=corpus, mode='lines+markers'), row=i, col=1)

#fig.update_traces(side='positive', orientation='h', width=2, points=False)
fig.update_layout(**STD_LAYOUT, showlegend=False, height=800, width=300)
fig.update_xaxes(gridcolor='lightgrey', zerolinecolor='lightgrey', tickmode='array', tickvals= [-12, -6, 0, 6, 12, 18],
    ticktext = ["Dbb", "Gb", "C", "F#", "B#", "E##"],)
fig.update_yaxes(showgrid=False, zeroline=False)
fig.write_image("/home/hentsche/Documents/phd/romantic_piano_corpus_report/figures/tpc_line_per_corpus.png", scale=2)
fig.show()

### Notes and staves

In [ ]:
print("Distribution of notes over staves:")
all_notes.staff.value_counts()

In [ ]:
print("Distribution of notes over staves for all pieces with more than two staves\n")
for group, df in all_notes.groupby(level=[0,1]):
    if (df.staff > 2).any():
        print(group)
        print(df.staff.value_counts().to_dict())

In [ ]:
all_notes[all_notes.staff > 2].groupby(level=[0,1]).staff.value_counts()

## Harmony labels

All symbols, independent of the local key (the mode of which changes their semantics).

In [ ]:
all_annotations = annotated.get_facet('expanded')
all_annotations.head()

In [ ]:
no_chord = all_annotations.root.isna()
print(f"Concatenated annotation tables contains {all_annotations.shape[0]} rows. {no_chord.sum()} of them are not chords. Their values are:")
all_annotations.label[no_chord].value_counts(dropna=False).to_dict()

In [ ]:
all_chords = all_annotations[~no_chord]
print(f"Corpus contains {all_chords.shape[0]} tokens and {len(all_chords.chord.unique())} types over {len(all_chords.groupby(level=[0,1]))} documents.")

In [ ]:
#from ms3 import write_tsv
#write_tsv(all_annotations[all_annotations.pedalend.notna()], './issues/pedalpoints.tsv', pre_process=False)

## Corpus summary

In [ ]:
summary = all_metadata.set_index('fnames', append=True).reset_index(level=[1,2], drop=True)
if selected == annotated:
    summary = summary[summary.label_count > 0].copy()
summary.index = summary.index.rename(['corpus', 'fname'])
summary.length_qb = all_measures.groupby(level=[0,1]).act_dur.sum() * 4.0
summary = pd.concat([summary,
                     all_notes.groupby(level=[0,1]).size().rename('notes'),
                    ], axis=1)
summary.groupby(level=0).describe().dropna(axis=1, how='all')

In [ ]:
corpus_metadata = summary.groupby(level=0)
n_pieces = corpus_metadata.size().rename('pieces')
absolute_numbers = dict(
    measures = corpus_metadata.last_mn.sum(),
    length = corpus_metadata.length_qb.sum(),
    notes = corpus_metadata.notes.sum(),
    labels = corpus_metadata.label_count.sum(),
)
absolute = pd.DataFrame.from_dict(absolute_numbers)
relative = absolute.div(n_pieces, axis=0).astype(float).round(1)
complete_summary = pd.concat([pd.concat([n_pieces, absolute], axis=1), relative], axis=1, keys=['absolute', 'per piece'])
complete_summary

# Harmony labels
## Unigrams
For computing unigram statistics, the tokens need to be grouped by their occurrence within a major or a minor key because this changes their meaning. To that aim, the annotated corpus needs to be sliced into contiguous localkey segments which are then grouped into a major (`is_minor=False`) and a minor group.

In [ ]:
localkey_slices = LocalKeySlicer().process_data(annotated)

In [ ]:
mode_slices = ModeGrouper().process_data(localkey_slices)

### Whole dataset

In [ ]:
unigrams = ChordSymbolUnigrams(once_per_group=True).process_data(mode_slices)

In [ ]:
unigrams.get()

In [ ]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in unigrams.iter():
    print(f"{modes[is_minor]} UNIGRAMS\n{ugs.shape[0]} types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

### Per corpus

In [ ]:
corpus_wise_unigrams = Pipeline([CorpusGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [ ]:
corpus_wise_unigrams.get()

In [ ]:
for (is_minor, corpus_name), ugs in corpus_wise_unigrams.iter():
    print(f"{corpus_name} {modes[is_minor]} unigrams ({ugs.shape[0]} types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

In [ ]:
types_shared_between_corpora = {}
for (is_minor, corpus_name), ugs in corpus_wise_unigrams.iter():
    if is_minor in types_shared_between_corpora:
        types_shared_between_corpora[is_minor] = types_shared_between_corpora[is_minor].intersection(ugs.index) 
    else:
        types_shared_between_corpora[is_minor] = set(ugs.index)
types_shared_between_corpora = {k: sorted(v, key=lambda x: unigrams.get()[(k, x)], reverse=True) for k, v in types_shared_between_corpora.items()}
n_types = {k: len(v) for k, v in types_shared_between_corpora.items()}
print(f"Chords which occur in all corpora, sorted by descending global frequency:\n{types_shared_between_corpora}\nCounts: {n_types}")

### Per piece

In [ ]:
piece_wise_unigrams = Pipeline([PieceGrouper(), ChordSymbolUnigrams(once_per_group=True)]).process_data(mode_slices)

In [ ]:
piece_wise_unigrams.get()

In [ ]:
types_shared_between_pieces = {}
for (is_minor, corpus_name), ugs in piece_wise_unigrams.iter():
    if is_minor in types_shared_between_pieces:
        types_shared_between_pieces[is_minor] = types_shared_between_pieces[is_minor].intersection(ugs.index) 
    else:
        types_shared_between_pieces[is_minor] = set(ugs.index)
print(types_shared_between_pieces)

## Bigrams

### Whole dataset

In [ ]:
bigrams = ChordSymbolBigrams(once_per_group=True).process_data(mode_slices)

In [ ]:
bigrams.get()

In [ ]:
modes = {True: 'MINOR', False: 'MAJOR'}
for (is_minor,), ugs in bigrams.iter():
    print(f"{modes[is_minor]} BIGRAMS\n{ugs.shape[0]} transition types, {ugs.sum()} tokens")
    print(ugs.head(20).to_string())

### Per corpus

In [ ]:
corpus_wise_bigrams = Pipeline([CorpusGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

In [ ]:
corpus_wise_bigrams.get()

In [ ]:
for (is_minor, corpus_name), ugs in corpus_wise_bigrams.iter():
    print(f"{corpus_name} {modes[is_minor]} bigrams ({ugs.shape[0]} transition types, {ugs.sum()} tokens)")
    print(ugs.head(5).to_string())

In [ ]:
normalized_corpus_unigrams = {group: (100 * ugs / ugs.sum()).round(1).rename("frequency") for group, ugs in corpus_wise_unigrams.iter()}

In [ ]:
transitions_from_shared_types = {
    False: {},
    True: {}
}
for (is_minor, corpus_name), bgs in corpus_wise_bigrams.iter():
    transitions_normalized_per_from = bgs.groupby(level="from").apply(lambda S: (100 * S / S.sum()).round(1))
    most_frequent_transition_per_from = transitions_normalized_per_from.rename('fraction').reset_index(level=1).groupby(level=0).nth(0)
    most_frequent_transition_per_shared = most_frequent_transition_per_from.loc[types_shared_between_corpora[is_minor]]
    unigram_frequency_of_shared = normalized_corpus_unigrams[(is_minor, corpus_name)].loc[types_shared_between_corpora[is_minor]]
    combined = pd.concat([unigram_frequency_of_shared, most_frequent_transition_per_shared], axis=1)
    transitions_from_shared_types[is_minor][corpus_name] = combined

In [ ]:
pd.concat(transitions_from_shared_types[False].values(), keys=transitions_from_shared_types[False].keys(), axis=1)

In [ ]:
pd.concat(transitions_from_shared_types[True].values(), keys=transitions_from_shared_types[False].keys(), axis=1)

### Per piece

In [ ]:
piece_wise_bigrams = Pipeline([PieceGrouper(), ChordSymbolBigrams(once_per_group=True)]).process_data(mode_slices)

In [ ]:
piece_wise_bigrams.get()